# 基本思路：
    1、获取stockbasic和stockearningreport的全量数据
    2、stockbasic根据所在市场进行过滤，只要主板和中小板的
    3、stockearingreport进行筛选，条件为：
        3.1 基于2中的主板和中小板数据
        3.2 只要最近3年的数据
        3.3 仅要年报（1231）这一天
    4、完成后，stockearningreport根据code进行聚类，并获取net_proft_yoy_raise的最小，均值和最大值
    5、根据group后的net_proft_yoy_raise最小值进行筛选，暂定条件为大于30%

In [1]:
from sqlalchemy.orm import Session
from dataUpdate import engine
from dataUpdate.model.basicinfo import StockEarningReport,StockBasic
import pandas as pd

In [2]:
sess = Session(engine)

In [3]:
# 1、获取所有报表是数据
sql = sess.query(StockEarningReport)
res_df = pd.read_sql(sql.statement,sql.session.bind)
res_df.head(3)

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,year,date
0,1,000617,-0.04,3.046625e+08,10.749529,-47.6992,-10572685.25,-134.422353,-231.3498,-1.27,-0.100000,3.111316,2010,0331
1,2,000757,-0.08,1.319204e+08,101.189055,16.3653,-25948235.48,16.468424,-146.0735,0.00,-0.042300,12.714494,2010,0331
2,3,301041,NaN,7.030412e+07,NaN,NaN,4112270.94,NaN,NaN,NaN,-0.169109,15.906955,2010,0331


In [14]:
# 2、获取主板+中小板数据
sql2 = sess.query(StockBasic.symbol,StockBasic.name,StockBasic.circulating_market_cap,StockBasic.sw_l1).filter(StockBasic.market.in_(['主板','中小板']))
res_df2 = pd.read_sql(sql2.statement,sql2.session.bind)
res_df2.head(3)

,symbol,name,circulating_market_cap,sw_l1
0,600000,浦发银行,3137.740,银行I
1,600004,白云机场,253.492,交通运输I
2,600006,东风汽车,128.400,汽车I


In [5]:
# 3、仅获取主板和中小板的，3年年份的年报数据
annul_report = res_df[(res_df['year'].isin([2018,2019,2020])) & (res_df['date']=='1231') & (res_df['code'].isin(res_df2['symbol']))]
annul_report.shape

(9327, 14)

In [6]:
annul_report[annul_report['code']=='601919']

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,year,date
192670,192671,601919,0.12,1.208295e+11,33.566485,4.4383,1.230026e+09,-53.792034,-55.2878,5.52,0.795865,8.462042,2018,1231
225298,225299,601919,0.56,1.510567e+11,25.016363,-0.1942,6.764105e+09,449.920000,428.7379,21.57,1.729460,10.742339,2019,1231
258685,258686,601919,0.81,1.712588e+11,13.373888,22.6643,9.927098e+09,46.760000,122.8038,25.05,3.673106,14.208792,2020,1231


In [7]:
# 4、根据code进行groupby，并计算年净利润增长率的最小，均值和最大值
annul_report_groupby = annul_report.groupby('code').agg({'net_profit_yoy_rise':['min','mean','max']})

In [17]:
# 5、strategy1: 最低净利润增长也要大于30
res1 = annul_report_groupby[annul_report_groupby['net_profit_yoy_rise']['min']>20]

In [18]:
res1.shape

(197, 3)

In [19]:
res1.head()

net_profit_yoy_rise                        
                       min        mean         max
code                                              
000009           40.860000   73.700000  119.750000
000020           25.090000  109.651962  238.155887
000037           21.061024   69.170341  157.120000
000049           25.120000   30.677186   33.551559
000090           20.220000   36.100641   58.170000

In [20]:
res2 = pd.merge(left=res1,right=res_df2,left_on='code',right_on='symbol',how='inner')

/Users/roland/anaconda3/envs/py374/lib/python3.7/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [21]:
res2.sort_values(by='circulating_market_cap',inplace=True,ascending=False)
res2.head()

,"(net_profit_yoy_rise, min)","(net_profit_yoy_rise, mean)","(net_profit_yoy_rise, max)",symbol,name,circulating_market_cap,sw_l1
144,22.285194,34.811731,49.580000,601888,中国国旅,7572.87,休闲服务I
8,29.380000,32.939113,36.267338,000568,泸州老窖,4597.78,食品饮料I
111,36.250000,103.936743,192.330230,600031,三一重工,4066.41,机械设备I
132,28.630000,46.341950,56.390000,600809,山西汾酒,3840.38,食品饮料I
67,53.280000,62.486099,73.130000,002475,立讯精密,3701.54,电子I


In [22]:
res2['sw_l1'].value_counts()

机械设备I    25
化工I      24
医药生物I    18
电子I      16
公用事业I    13
计算机I     12
食品饮料I    11
电气设备I    10
房地产I      9
轻工制造I     9
建筑装饰I     6
建筑材料I     6
有色金属I     5
传媒I       5
交通运输I     4
家用电器I     4
采掘I       3
汽车I       3
国防军工I     2
非银金融I     2
综合I       2
纺织服装I     2
商业贸易I     2
通信I       2
钢铁I       1
休闲服务I     1
Name: sw_l1, dtype: int64

In [24]:
res2[res2.sw_l1=='医药生物I']

,"(net_profit_yoy_rise, min)","(net_profit_yoy_rise, mean)","(net_profit_yoy_rise, max)",symbol,name,circulating_market_cap,sw_l1
119,20.250000,27.831366,41.624098,600436,片仔癀,2376.8300,医药生物I
14,52.050341,66.683447,76.360000,000661,长春高新,1784.0500,医药生物I
179,23.774388,123.818129,275.240000,603882,金域医学,764.3580,医药生物I
94,25.493984,28.394661,30.370000,002821,凯莱英,733.8050,医药生物I
185,30.580000,34.898652,41.290000,603939,益丰药房,545.9710,医药生物I
151,41.718019,58.342673,68.670000,603127,昭衍新药,366.1030,医药生物I
171,33.250000,36.953678,42.500000,603707,健友股份,317.7650,医药生物I
17,44.418654,47.112885,49.340000,000739,普洛药业,278.3380,医药生物I
125,23.060000,29.145638,36.326914,600529,山东药玻,245.6620,医药生物I
97,25.320000,27.039769,30.120000,002901,大博医疗,188.6900,医药生物I
